# Simple Entry Removal Algorithm 2 (SER 2)

In [1]:
import numpy as np
import datetime
import random
import pandas as pd
import json

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [13]:
def add_up_a_decomposition_float(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=float)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# The SER 2 algorithm

In [3]:
# This function is used in the SER 2 function.
# residue_matrix_R is a np.ndarray.
# chosen_entry_pos_each_col is a 1-by-PBN_col_sum np.ndarray.
# PBN_col_num is the number of columns present in residue_matrix_R

def choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num):
    col_counter = 0
    chosen_entry_xi = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
    
    for col_counter in range(PBN_col_num):
        current_entry = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
        if current_entry < chosen_entry_xi:
            chosen_entry_xi = current_entry
    
    return chosen_entry_xi

In [4]:
def float_SER_2(PBN_matrix_P, stopping_error):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4

    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
#    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (np.linalg.norm(residue_matrix_R, ord='fro') >= stopping_error):
        k += 1
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai, np.linalg.norm(residue_matrix_R, ord='fro')

# Floating-Point Matrix Experiment

In [5]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

## $8 \times 8$ TPMs

In [23]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './8-by-8/dim8_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_2(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_2)/dim8_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_2)/dim8_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 01:26:43.839240

iteration 1
Time now is: 2025-04-12 01:26:43.924149

iteration 2
Time now is: 2025-04-12 01:26:43.950651

iteration 3
Time now is: 2025-04-12 01:26:43.973052

iteration 4
Time now is: 2025-04-12 01:26:43.990722

iteration 5
Time now is: 2025-04-12 01:26:44.008606

iteration 6
Time now is: 2025-04-12 01:26:44.034273

iteration 7
Time now is: 2025-04-12 01:26:44.055731

iteration 8
Time now is: 2025-04-12 01:26:44.074579

iteration 9
Time now is: 2025-04-12 01:26:44.094145



## $16 \times 16$ TPMs

In [43]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './16-by-16/dim16_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_2(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_2)/dim16_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_2)/dim16_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 01:54:43.248947

iteration 1
Time now is: 2025-04-12 01:54:43.253806

iteration 2
Time now is: 2025-04-12 01:54:43.257806

iteration 3
Time now is: 2025-04-12 01:54:43.262847

iteration 4
Time now is: 2025-04-12 01:54:43.267847

iteration 5
Time now is: 2025-04-12 01:54:43.271397

iteration 6
Time now is: 2025-04-12 01:54:43.274798

iteration 7
Time now is: 2025-04-12 01:54:43.277183

iteration 8
Time now is: 2025-04-12 01:54:43.280801

iteration 9
Time now is: 2025-04-12 01:54:43.284048



## $32 \times 32$ TPMs

In [42]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './32-by-32/dim32_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_2(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_2)/dim32_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_2)/dim32_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 01:54:21.434112

iteration 1
Time now is: 2025-04-12 01:54:21.443153

iteration 2
Time now is: 2025-04-12 01:54:21.449710

iteration 3
Time now is: 2025-04-12 01:54:21.455730

iteration 4
Time now is: 2025-04-12 01:54:21.463616

iteration 5
Time now is: 2025-04-12 01:54:21.470141

iteration 6
Time now is: 2025-04-12 01:54:21.483592

iteration 7
Time now is: 2025-04-12 01:54:21.498127

iteration 8
Time now is: 2025-04-12 01:54:21.514674

iteration 9
Time now is: 2025-04-12 01:54:21.523578



## $64 \times 64$ TPMs

In [39]:
stopping_error = 0.01

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    fTPM_filepath = './64-by-64/dim64_fTPM_' + str(k) + '.npy'
    fTPM = np.load(fTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom_and_error = float_SER_2(fTPM, stopping_error)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (float_SER_2)/dim64_decom_err_" + str(k) + ".json"
    time_filepath = "./raw data (float_SER_2)/dim64_time_" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom_and_error, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 01:52:45.367478

iteration 1
Time now is: 2025-04-12 01:52:45.388864

iteration 2
Time now is: 2025-04-12 01:52:45.420345

iteration 3
Time now is: 2025-04-12 01:52:45.450883

iteration 4
Time now is: 2025-04-12 01:52:45.489068

iteration 5
Time now is: 2025-04-12 01:52:45.526499

iteration 6
Time now is: 2025-04-12 01:52:45.559799

iteration 7
Time now is: 2025-04-12 01:52:45.585371

iteration 8
Time now is: 2025-04-12 01:52:45.617243

iteration 9
Time now is: 2025-04-12 01:52:45.647265



# Some random test code

In [15]:
test_matrix = np.random.exponential(size=(4, 4))

for k in range(4):
    test_matrix[:, k] = test_matrix[:, k] / np.sum(test_matrix[:, k])

In [16]:
np.sum(test_matrix, axis=0)

array([1., 1., 1., 1.])

In [17]:
K, x_list, BN_list, error = float_SER_2(test_matrix, 0.001)

In [18]:
K

12

In [19]:
error

0.000513309232779465

In [20]:
test_matrix

array([[0.30104727, 0.45703437, 0.00701529, 0.25848057],
       [0.14486512, 0.39931011, 0.53203814, 0.11791478],
       [0.48952471, 0.02263924, 0.22389972, 0.24989712],
       [0.0645629 , 0.12101628, 0.23704684, 0.37370753]])

In [22]:
add_up_a_decomposition_float(x_list, BN_list, 4, 4)

array([[0.30079062, 0.45703437, 0.00675864, 0.25848057],
       [0.14486512, 0.39905346, 0.53203814, 0.11765812],
       [0.48952471, 0.02263924, 0.22389972, 0.24989712],
       [0.0645629 , 0.12101628, 0.23704684, 0.37370753]])